# Experiment with `langchain`

In [1]:
from langchain.prompts import ChatPromptTemplate
import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    StoppingCriteria, 
    StoppingCriteriaList, 
    TextIteratorStreamer
    )

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from samples import samples

In [3]:
template_string = """
Instruct: Classify the following text in one of the following categories: ["support", "sales", "joke"]. Output only the name of the category.
+ "support" for customer support texts
+ "sales" for sales and comercial texts
+ "joke" for jokes, funny or comedy like texts
Text: {text}
Output:
""".strip()

In [4]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [5]:
prompt_template.messages[0].input_variables

['text']

In [6]:
prompt_template.format_messages(text="My package was lost")

[HumanMessage(content='Instruct: Classify the following text in one of the following categories: ["support", "sales", "joke"]. Output only the name of the category.\n+ "support" for customer support texts\n+ "sales" for sales and comercial texts\n+ "joke" for jokes, funny or comedy like texts\nText: My package was lost\nOutput:')]

In [7]:
class StopOnTokens(StoppingCriteria):
    """Stops the model if it produces an 'end of text' token"""
    def __call__(self, input_ids: torch.LongTensor, 
                 scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50256, 198] # <|endoftext|> and EOL
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

## Connect langchain and huggingface

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Your device is", device)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2", 
    device_map="auto", 
    torch_dtype="auto" if device == "cuda" else torch.float, 
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2", trust_remote_code=True)

Your device is cuda


Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00,  2.39it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200, stopping_criteria=[StopOnTokens()])
hf = HuggingFacePipeline(pipeline=pipe)

In [10]:
from random import choice

In [11]:
choice(list(samples.keys()))

'support'

In [24]:
chain = prompt_template | hf

category = choice(list(samples.keys()))
text = choice(samples[category])

print(chain.invoke({"text": text}))

 support



## Experiment with ECL

In [16]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [17]:
chain = (
    prompt_template
    | hf
    | StrOutputParser()
)

In [18]:
print(chain.invoke({"text": text}))

 joke



## Experiment with routing

In [19]:
support_template = """\
Instruction: You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. \
Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.
Query: {text}
Output:\
"""

sales_template = """\
Instruction: You are an aggressive salesperson. The user is looking for some information on products. \
Reply to their query by giving information on related products and showcasing how good they are and why they should buy them.
Query: {text}
Output:\
"""

joke_template = """\
Instruction: You are a comedian. The user want's to have some fun. Reply to their query in a funny way.
Query: {text}
Output:\
"""

general_template = """\
Instruction: Respond to the following query.
Query: {text}
Output:\
"""

support_chain = PromptTemplate.from_template(support_template) | hf
sales_chain = PromptTemplate.from_template(sales_template) | hf
joke_chain = PromptTemplate.from_template(joke_template) | hf
general_chain = PromptTemplate.from_template(general_template) | hf

In [25]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "support" in x["topic"].lower(), support_chain),
    (lambda x: "sales" in x["topic"].lower(), sales_chain),
    (lambda x: "joke" in x["topic"].lower(), joke_chain),
    general_chain,
)

In [26]:
full_chain = {"topic": chain, "text": lambda x: x["text"]} | branch

In [27]:
print(full_chain.invoke({"text": text}))

 Hello! Thank you for reaching out to us. We apologize for any inconvenience you may have experienced. We do have a comprehensive FAQ section on our website that addresses common customer queries. Please visit our FAQ page at www.example.com/faq for more information. If you have any further questions or concerns, please don't hesitate to contact us again. We're here to help!



In [41]:
for _ in range(10):
    topic = choice(list(samples.keys()))
    text = choice(samples[topic])
    print(topic.upper(), text)

    print(full_chain.invoke({"text": text}))
    print("========================================================")

SUPPORT Is there a customer support hotline I can call for immediate help?
 I'm sorry to hear that you're experiencing difficulties. Yes, there is a customer support hotline available for immediate assistance. You can reach us at 1-800-123-4567. Our team is here to help you resolve any issues you may be facing.

SUPPORT Hi, I have an issue with my order. It hasn't arrived, and the delivery date has passed.
 Hello! I'm sorry to hear that you haven't received your order yet. I understand how frustrating this can be. Let me check the status of your order and see what I can do to help. Could you please provide me with your order number?

SUPPORT What is the return process? I'm not satisfied with my purchase.
 I'm sorry to hear that you're not satisfied with your purchase. I understand how frustrating that can be. Our return process is quite simple. All you need to do is fill out our online return form and we'll take care of the rest. We'll send you a prepaid shipping label and instructions

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Hello! I'm sorry to hear that you haven't received your order yet. I understand how frustrating this can be. Let me check the status of your order and see what I can do to help. Could you please provide me with your order number?

SALES I'm a small business owner looking to purchase computers in bulk. Are there discounts available?


/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Absolutely! We have a wide range of computers that are perfect for small businesses. Our bulk purchase options offer significant discounts, and our customer service team is here to help you find the perfect solution for your business. Don't miss out on this opportunity to save money and get the best products for your business!

JOKE What do you call a robot that likes to take naps? A recharger!


...

 I'm sorry to hear that you're not satisfied with the resolution of your issue. I understand how frustrating that can be. If you feel that your issue is not being addressed satisfactorily, you can escalate it to a supervisor. Please let me know the details of your issue, and I'll be happy to assist you in escalating it to the appropriate person.

SUPPORT What is your policy on returns for international orders?


/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Hello, thank you for contacting us. We appreciate your interest in our products and we apologize for any inconvenience caused by the international shipping. Our policy on returns for international orders is that we accept them within 30 days of delivery, as long as the item is in its original condition and packaging. However, we cannot issue refunds or exchanges for international orders, as we are not responsible for the customs fees or the delivery charges. We hope this information helps and we thank you for your patience and understanding.

SALES Can you provide details on the warranty coverage for your electronics?


/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 I'm sorry to hear that you're having trouble with your electronics. I'd be happy to provide you with information on our warranty coverage. Could you please provide me with the make and model of the product you're inquiring about?



## Adding memory to the chat system

One thing that's now left is memory of the chat turns. You can see a working example [here](https://python.langchain.com/docs/modules/memory/adding_memory).

In [28]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [76]:
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

In [77]:
llm_chain = LLMChain(
    llm=hf,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [32]:
llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hello! How can I help you today?

Human: Hi there my friend
Chatbot:

> Finished chain.


' Hi! What can I do for you?\n'

## Putting it all together

> I think for this section, the [memory chain with multiple inputs](https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs) is a good resource.

Now, in order to put it all together (conditional branching and conversation memory), we first need to modify a bit our prompts from the _routing_ section. They were meant to be used standalone and they make use of the `Instruction/Output` template from `phi-2`. When we add the conversation bits, this doesn't make sense anymore.

We're going to do this in 3 steps:
1. Create the classification chain
2. Add the branch template
3. Add the conversation memory

### First, create the initial classification template

We change `text` variable to `human_input` as this is the name we'll use for the chat at the end.

In [56]:
classification_template = """
Instruct: Classify the following text in one of the following categories: ["support", "sales", "joke"]. Output only the name of the category.
+ "support" for customer support texts
+ "sales" for sales and comercial texts
+ "joke" for jokes, funny or comedy like texts
Text: {human_input}
Output:
""".strip()

In [57]:
classification_prompt = ChatPromptTemplate.from_template(classification_template)

In [236]:
classification_chain = (
    classification_prompt
    | hf
    | StrOutputParser()
)

In [237]:
print(classification_chain.invoke({"human_input": "Can I track my order? I'm eager to know its status"}))

 support



/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


### Second, add the branch template

The branch template gives the right instructions for the chatbot.

In [238]:
support_instructions = """\
You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. \
Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.\
"""

sales_instructions = """\
Instruction: You are an aggressive salesperson. The user is looking for some information on products. \
Reply to their query by giving information on related products and showcasing how good they are and why they should buy them.\
"""

joke_instructions = """\
Instruction: You are a comedian. The user want's to have some fun. Reply to their query in a funny way.\
"""

general_instructions = """\
Instruction: Respond to the following query.\
"""

support_chain = PromptTemplate.from_template(support_instructions)
sales_chain = PromptTemplate.from_template(sales_instructions)
joke_chain = PromptTemplate.from_template(joke_instructions)
general_chain = PromptTemplate.from_template(general_instructions)

In [249]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "support" in x["topic"].lower(), support_chain),
    (lambda x: "sales" in x["topic"].lower(), sales_chain),
    (lambda x: "joke" in x["topic"].lower(), joke_chain),
    general_chain,
) | RuStringPromptValuennableLambda(lambda x: x.text)

In [250]:
branch_chain = {"topic": classification_chain, "human_input": lambda x: x["human_input"]} | branch

In [251]:
response = branch_chain.invoke({"human_input": "Can I track my order? I'm eager to know its status"})

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [252]:
response

"You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary."

### Third and final, add the memory and chat template

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [274]:
template = """\
You are a chatbot having a conversation with a human. Follow the given instructions to reply to the Human message below.

Instructions:{instructions}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["instructions", "chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(
    return_messages=False,
    ai_prefix="Chatbot", 
    human_prefix="Human",
    memory_key="chat_history"
)

In [275]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [276]:
def print_prompt(prompt):
    print("---FULL PROMPT---")
    print(prompt.text)
    print("---END  PROMPT---")
    return prompt

chat_chain = (
    {
        "human_input": lambda x: x["human_input"], 
        "instructions": lambda x: branch_chain,
        "chat_history": (RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")),
    } | prompt | RunnableLambda(print_prompt) | hf
)

In [257]:
input_ = {"human_input": "Can I track my order? I'm eager to know its status"}

In [270]:
input_ = {"human_input": "My tracking number is 1"}

In [271]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

response = chat_chain.invoke(
    input_
    #, config={'callbacks': [ConsoleCallbackHandler()]}
)
memory.save_context({"input": input_['human_input']}, {"ouput": response})
response

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


---FULL PROMPT---
You are a chatbot having a conversation with a human. Follow the given instructions to reply to the Human message below.

Instructions:You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.


Human: My tracking number is 1
Chatbot:
---END  PROMPT---


" I'm sorry to hear that you are having trouble with your order. Can you please provide me with your order number so that I can look into this for you?\n"

In [273]:
print(memory.load_memory_variables({})['history'])

Human: Can I track my order? I'm eager to know its status
AI: Could you please provide me with your order number so that I can look into it for you?
Human: Can I track my order? I'm eager to know its status
AI:  I'm sorry to hear that you're eager to know the status of your order. I understand how important it is to you. Could you please provide me with your order number so that I can look into it for you?

Human: My tracking number is 1
AI:  I'm sorry to hear that you are having trouble with your order. Can you please provide me with your order number so that I can look into this for you?



In [259]:
chat_chain.invoke(input_)

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


---FULL PROMPT---
You are a chatbot having a conversation with a human. Follow the given instructions to reply to the Human message below.

Instructions:You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.


Human: Can I track my order? I'm eager to know its status
Chatbot:
---END  PROMPT---


" I'm sorry to hear that you're eager to know the status of your order. I understand how important it is to you. Could you please provide me with your order number so that I can look into it for you?\n"

In [140]:
(RunnableLambda(memory.load_memory_variables) | itemgetter("history")).invoke(input_)

''

In [261]:
memory = ConversationBufferMemory(return_messages=False)
memory.save_context(
    {"input": input_['human_input']}, 
    {"output": "Could you please provide me with your order number so that I can look into it for you?"})

In [262]:
print(memory.load_memory_variables({})['history'])

Human: Can I track my order? I'm eager to know its status
AI: Could you please provide me with your order number so that I can look into it for you?


In [263]:
print((
    RunnableParallel(
        human_input=lambda x: x["human_input"], 
        instructions=lambda x: branch_chain,
        history= (RunnableLambda(memory.load_memory_variables) | itemgetter("history")),
    ) | prompt
).invoke(input_).text)

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


KeyError: "Input to PromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'human_input', 'instructions'] Received: ['human_input', 'instructions', 'history']"

In [144]:
from langchain.prompts.base import StringPromptValue

In [145]:
hf.invoke(
    {'human_input': "Can I track my order? I'm eager to know its status",
 'instructions': StringPromptValue(text="You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary."),
 'history': ''}
)

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [102]:
memory.load_memory_variables({})

{'history': ''}

In [111]:
branch_chain.invoke(input_) | itemgetter("text")

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


TypeError: unsupported operand type(s) for |: 'StringPromptValue' and 'operator.itemgetter'

In [122]:
RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
        ).invoke(input_)

{'human_input': "Can I track my order? I'm eager to know its status",
 'history': ''}

In [116]:
input_

{'human_input': "Can I track my order? I'm eager to know its status"}

In [120]:
RunnableParallel(
    history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
).invoke(input_)

{'history': ''}

In [123]:
(RunnableLambda(memory.load_memory_variables) | itemgetter("history")).invoke(input_)

''

In [119]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
)

runnable.invoke(input_)

{'passed': {'human_input': "Can I track my order? I'm eager to know its status"}}